In [1]:
!pip install javascript

  Using cached javascript-1%211.2.6-py3-none-any.whl.metadata (14 kB)
Using cached javascript-1%211.2.6-py3-none-any.whl (34 kB)


In [ ]:
from javascript import require, On, Once, AsyncTask, once, off
mineflayer = require('mineflayer')
random_number = id([]) % 1000 # Give us a random number upto 1000
BOT_USERNAME = f'colab_{random_number}'

bot = mineflayer.createBot({ 'host': '127.0.0.1', 'port': 25565, 'username': BOT_USERNAME, 'hideErrors': False })

# The spawn event
once(bot, 'login')
bot.chat('I spawned')
bot.entity.position
@On(bot, 'playerJoin')
def end(this, player):
  bot.chat('Someone joined!')

@On(bot, 'chat')
def onChat(this, user, message, *rest):
  print(f'{user} said "{message}"')

  # If the message contains stop, remove the event listener and stop logging.
  if 'stop' in message:
    off(bot, 'chat', onChat)

@On(bot, 'chat')
def breakListener(this, sender, message, *args):
  if sender and (sender != BOT_USERNAME):
    if 'break' in message:
      pos = bot.entity.position.offset(0, -1, 0)
      blockUnder = bot.blockAt(pos)
      if bot.canDigBlock(blockUnder):
        bot.chat(f"I'm breaking the '{blockUnder.name}' block underneath")
        # The start=True parameter means to immediately invoke the function underneath
        # If left blank, you can start it with the `start()` function later on.
        try:
          @AsyncTask(start=True)
          def break_block(task):
            bot.dig(blockUnder)
          bot.chat('I started digging!')
        except Exception as e:
          bot.chat(f"I had an error {e}")
      else:
        bot.chat(f"I can't break the '{blockUnder.name}' block underneath")
    if 'stop' in message:
      off(bot, 'chat', breakListener)
      
pathfinder = require('mineflayer-pathfinder')
bot.loadPlugin(pathfinder.pathfinder)
# Create a new minecraft-data instance with the bot's version
mcData = require('minecraft-data')(bot.version)
# Create a new movements class
movements = pathfinder.Movements(bot, mcData)
# How far to be from the goal
RANGE_GOAL = 1

bot.removeAllListeners('chat')
@On(bot, 'chat')
def handleMsg(this, sender, message, *args):
  if sender and (sender != BOT_USERNAME):
    bot.chat('Hi, you said ' + message)
    if 'come' in message:
      player = bot.players[sender]
      target = player.entity
      if not target:
        bot.chat("I don't see you !")
        return
      pos = target.position
      bot.pathfinder.setMovements(movements)
      bot.pathfinder.setGoal(pathfinder.goals.GoalNear(pos.x, pos.y, pos.z, RANGE_GOAL))
    if 'stop' in message:
      off(bot, 'chat', handleMsg)

import matplotlib.pyplot as plt
figure = plt.figure()
axes = figure.add_axes([0,0,1,1])
Vec3 = require('vec3').Vec3

columns = bot.world.getColumns()
block_freqs = {}
for c in range(0, 3): # iterate through some of the loaded chunk columns
  cc = columns[c].column
  for y in range(1, 40):
    for x in range(1, 16):
      for z in range(1, 16):
        block = cc.getBlock(Vec3(x, y, z))
        if block.name in block_freqs:
          block_freqs[block.name] += 1
        else:
          block_freqs[block.name] = 1

print(block_freqs)
axes.bar(block_freqs.keys(), block_freqs.values())
plt.xticks(rotation=45)
plt.show()

bot.quit()

# 🧩 MineEvac Simulation Process — Mathematical Formulation

## 1️⃣ Environment Initialization

A building layout  
$$
\ell \in \mathcal{L}
$$
is defined as
$$
\ell = \big(\mathcal{V}(\ell), \mathcal{E}(\ell), \mathcal{S}(\ell), N_F(\ell)\big)
$$

where:
- $\mathcal{V}(\ell) = \{v_1, \dots, v_K\}$ : set of rooms  
- $\mathcal{E}(\ell) = \{e_1, \dots, e_M\}$ : set of exits  
- $\mathcal{S}(\ell)$ : set of stairwells  
- $N_F(\ell)$ : number of floors  

Occupants:
$$
\mathcal{O} = \{ o_1, o_2, \dots, o_{N_O} \}, \quad 
\mathbf{p}_o(0) \in \mathcal{V}(\ell)
$$
denotes the room where occupant $o$ initially resides.

Responders:
$$
\mathcal{R} = \{ r_1, r_2, \dots, r_{N_R} \}, \quad 
\mathbf{p}_r(0) = \mathbf{p}^{\mathrm{init}}_r
$$

The initial state of the environment:
$$
\mathbf{s}(0) = \big\{ \mathbf{p}_r(0), \mathbf{p}_o(0), E_v(0), H_v(0) 
\mid r\in\mathcal{R}, o\in\mathcal{O}, v\in\mathcal{V}(\ell) \big\}
$$

---

## 2️⃣ Time Evolution (Evacuation Dynamics)

Time is discretized with step size $\Delta t$.  
At each step $t \to t+1$:

1. Each responder selects an action $\mathbf{u}_r(t)$.  
2. The environment updates according to
   $$
   \mathbf{s}(t+1) = f\big(\mathbf{s}(t), \{ \mathbf{u}_r(t)\}_{r\in\mathcal{R}} \big)
   $$
   where $f(\cdot)$ is the simulation transition function.

Occupant motion rule:
$$
\mathbf{p}_o(t+1) = \mathbf{p}_o(t) + v_o \cdot \hat{\mathbf{d}}_o(t)
$$
where $\hat{\mathbf{d}}_o(t)$ is a unit vector toward the nearest exit.

Responder rule (fixed visiting plan):  
Each responder follows a room sequence
$$
\pi_r = (v^{(r)}_1, v^{(r)}_2, \dots)
$$
When entering a new room $v^{(r)}_i$,
$$
E_{v^{(r)}_i}(t) \gets 1, \quad H_{v^{(r)}_i}(t) \gets 0
$$

---

## 3️⃣ Sweep Order Formation

Define the **visit time** of room $v$:
$$
\tau(v) = \min \{ t \mid \exists r \in \mathcal{R}, \ \mathbf{p}_r(t) \in v \}
$$

Then sort rooms by visit time:
$$
\sigma = \mathrm{sort}\big( \{ (v, \tau(v)) \mid v \in \mathcal{V}(\ell) \}, \text{by } \tau(v) \big)
$$

Hence the **sweep order** is
$$
\sigma = (v_{(1)}, v_{(2)}, \dots, v_{(K)}), 
\quad \tau(v_{(1)}) < \tau(v_{(2)}) < \dots
$$

---

## 4️⃣ Evacuation Completion and Time

Each occupant’s evacuation completion time:
$$
T_o = \min \{ t \mid \mathbf{p}_o(t) \in \mathcal{E}(\ell) \}
$$

Overall evacuation time:
$$
T_{\mathrm{evac}} = \max_{o \in \mathcal{O}} T_o
$$

The simulation ends when all occupants satisfy  
$\mathbf{p}_o(T_{\mathrm{evac}}) \in \mathcal{E}(\ell)$.

---

## 5️⃣ Simulation Output

The simulation returns:
$$
\mathcal{R}_{\mathrm{sim}} 
= (\sigma, T_{\mathrm{evac}}, \{\mathbf{p}_r(t), \mathbf{p}_o(t)\}_{t=0}^{T_{\mathrm{evac}}})
$$

Where:
- $\sigma$: sweep order  
- $T_{\mathrm{evac}}$: total evacuation time  
- trajectories of all responders and occupants  

---

## 6️⃣ Symbol ↔ Code Mapping

| 数学符号 | 代码变量 | 说明 |
|-----------|------------|------|
| $\mathbf{s}(t)$ | `obs, info` | MineStudio environment state (observation + info) |
| $\mathbf{u}_r(t)$ | `action` | responder action |
| $f(\cdot)$ | `sim.step(action)` | environment transition function |
| $\sigma$ | `sweep_order` | ordered list of visited rooms |
| $T_{\mathrm{evac}}$ | `evac_time` | total evacuation time |
| $\mathbf{p}_r(t), \mathbf{p}_o(t)$ | `agent.position` | real-time positions |

---

## 🧠 Summary

This process defines a deterministic mapping:
$$
F_{\text{sim}}: (\ell, \mathcal{O}, \mathcal{R}, \{\pi_r\})
\longrightarrow (\sigma, T_{\mathrm{evac}})
$$
meaning:  
given a building layout, occupants, responders, and their sweep paths,  
the simulation deterministically produces the **room sweep order** and the **total evacuation time**.
